In [1]:
from datasets import load_dataset
data_files = {"train": "/home/qxy699/Data/WHO_representative_random/who_dataset_training.csv", 
              "test": "/home/qxy699/Data/WHO_representative_random/who_dataset_test.csv"}
# \t is the tab character in Python
sars_dataset = load_dataset("csv", data_files=data_files)

/home/qxy699/hugging_face/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_training_corpus():
    dataset = sars_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["sequence"]

In [3]:
training_corpus = get_training_corpus()

In [2]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [6]:
'''
vocab_size = how many unique tokens the tokenizer should recognize and 
maintain in its dictionary.  a larger vocabulary can capture more specific or rare words, 
potentially improving the model's ability to understand nuanced text, but it may also 
require more memory and computational resources. Conversely, a smaller vocabulary size 
might lead to a more generalized model that is efficient but less capable of understanding 
specific or rare terms, as these will be grouped into more general tokens or categorized 
as unknown.
'''
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [23]:
example = '''atcgaattccggatgccaatcgatcggtacgaatgtcatgatgcat'''

old_tokens = old_tokenizer.tokenize(example)
old_tokens

['at',
 'c',
 'ga',
 'att',
 'cc',
 'gg',
 'at',
 'g',
 'cca',
 'at',
 'c',
 'gat',
 'c',
 'gg',
 't',
 'ac',
 'ga',
 'at',
 'gt',
 'cat',
 'gat',
 'g',
 'cat']

In [24]:
example = '''atcgaattccggatgccaatcgatcggtacgaatgtcatgatgcat'''

tokens = tokenizer.tokenize(example)
tokens

['atcg',
 'aattcc',
 'gg',
 'atgcc',
 'aatcg',
 'atcgg',
 'tacg',
 'aatgtc',
 'atg',
 'atgc',
 'at']

In [18]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

10
22


In [25]:
tokenizer.save_pretrained("code-SARS-tokenizer")

('code-SARS-tokenizer/tokenizer_config.json',
 'code-SARS-tokenizer/special_tokens_map.json',
 'code-SARS-tokenizer/vocab.json',
 'code-SARS-tokenizer/merges.txt',
 'code-SARS-tokenizer/added_tokens.json',
 'code-SARS-tokenizer/tokenizer.json')

In [26]:
#load again
tokenizer = AutoTokenizer.from_pretrained("code-SARS-tokenizer")

In [32]:
sequence = "atcgaa", "ttccgg", "atgcca", "atcgat", "cggta cgaatg tcatga tgcat"
inputs = old_tokenizer(
    sequence, 
    truncation=True, 
    max_length=6, stride=2,
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(old_tokenizer.decode(ids))

atcgaa
ttccgg
atgcca
atcgat
cggta cgaat
gaatg tcatga
catga tgcat


In [33]:
inputs

{'input_ids': [[265, 66, 4908, 64], [926, 535, 1130], [265, 70, 13227], [265, 66, 41268], [66, 1130, 8326, 269, 4908, 265], [4908, 265, 70, 256, 9246, 4908], [9246, 4908, 256, 70, 9246]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 1, 2, 3, 4, 4, 4]}

In [34]:
sequence = "atcgaa", "ttccgg", "atgcca", "atcgat", "cggta cgaatg tcatga tgcat"
inputs = old_tokenizer(
    sequence, 
    truncation=True, 
    # max_length=6, stride=2,
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(old_tokenizer.decode(ids))

atcgaa
ttccgg
atgcca
atcgat
cggta cgaatg tcatga tgcat


In [35]:
inputs

{'input_ids': [[265, 66, 4908, 64], [926, 535, 1130], [265, 70, 13227], [265, 66, 41268], [66, 1130, 8326, 269, 4908, 265, 70, 256, 9246, 4908, 256, 70, 9246]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 1, 2, 3, 4]}